In [1]:
import gurobipy as gp
from gurobipy import GRB
import json

# Data Import
with open('export.json') as json_file:
    data = json.load(json_file)


# Declaracion de Sets
etapas = list(range(15))

cont = []     # Contenedores: C
c_llegan = [] # Contenedores que llegan: C+
c_salen = []  # Contenedores que salen: C-
c_fijos = []  # Contenedores que no de deben mover
c_libres = [] # Contenedores que se van a tener que relocar

c_mueven = [] # Contenedores que salen o lle,gan (ordenados): Cm


positions =  []    # Posiciones en el patio: P
lp = ['arrival']   # Posición de llegada
lm = ['retrival']  # Posición de salida

all_pos = [] # (auxiliar) Positions + lm + lp
fix_pos = [] # (auxiliar) Posiciones que no se moveran
mov_pos = [] # (auxiliar) Posiciones que se moveran

U = {}          # Posición bajo de i in P: U[i]
B = {}          # Posiones que bloquean a i in I: B[i]
B_I = {}        # Posiciones bloqueadas por i in I: B^{-1}[i]

#Declaración de Parámetros
Y0 = {}         # Posición inicial de cada Contenedor, vale 1 si el cont c esta en 
                # la posicion p en t=0 {('cont', 'positions'=: 1}

T = {}          # Tiempo de salida del contenedor c {'cont': int}
T_In = {}       # (auxiliar) Tiempo de llegada 

# Deficion de Sets [tc, pos_ini, status]
for c,[tc,pos,status] in data['contenedores'].items(): #val=[t_salida, posicion, status]
    ##### TEST 1/3 BORRAR DESPUES
    #if pos[1] == 'D' or pos[4] != '-':
    #    print('Removido {} de {}'.format(c, pos))
    #   continue
    ###### FIN TEST ######3    
    cont.append(c)
    T[c] = tc
    Y0[(c, pos)] = 1
    if status == 'fijo':
        c_fijos.append(c)
    elif status == 'sale':
        c_salen.append(c)
    else:
        c_libres.append(c)


for c,[tin, tc] in data['cont_llegadas'].items(): #val=[t_llegada, t_salida]
    cont.append(c)
    c_llegan.append(c)
    T[c] = tc
    T_In[c] = tin
    Y0[c,'arrival'] = 1

# Armamos la lista de los que se mueven
# Primero los que salen en con el tiempo de salida
c_mueven = []
aux_orden = []
for c in c_salen:
    c_mueven.append(c)
    aux_orden.append(T[c])
# Luego los que llegan con el tiempo de llegada
for c in c_llegan:
    c_mueven.append(c)
    aux_orden.append(T_In[c])
# Finalmente los ordenamos:

c_mueven = [i for _,i in sorted(zip(aux_orden,c_mueven))]


#print("======= TODOS =======")
#for c in cont:
#    print('{:9} : Tc={:5}'.format(c,T[c]))
#print("======= LLEGAN =======")
#for c in c_llegan:
#    print('{:9} : T_IN={:5} Tc={:5}'.format(c,T_In[c], T[c]))

#print("======= SALEN =======")
#for c in c_salen:
#    print('{:9} : Tc={:5}'.format(c,T[c]))
#print("======= MUEVEN =======")
#for c in c_mueven:
#    print('{:9}'.format(c))
#print("======= FIJOS =======")
#for c in c_fijos:
#    print('{:9}'.format(c))

print("Resumen contenedores")
print("-------------------------")
print("Total  (cont)     : {:5}".format(len(cont)))
print("Llegan (c_llegan) : {:5}".format(len(c_llegan)))
print("Salen  (c_salen)  : {:5}".format(len(c_salen)))
print("Mueven (c_mueven) : {:5}".format(len(c_mueven)))
print("Fijos  (c_fijos)  : {:5}".format(len(c_fijos)))
print("Libres (c_libres) : {:5}".format(len(c_libres)))
print("-------------------------")

Resumen contenedores
-------------------------
Total  (cont)     :   617
Llegan (c_llegan) :    26
Salen  (c_salen)  :    15
Mueven (c_mueven) :    41
Fijos  (c_fijos)  :   572
Libres (c_libres) :     4
-------------------------


In [2]:
# Leer archivo de posiciones

with open('export_posiciones.json') as json_file:
    data_pos = json.load(json_file)
    
for p in data_pos['posiciones']:
    
    ### TEST BORRAR #####
    """if p[1] == 'D' or p[4] != '-':
    #    print('Removido {}'.format(p))
        continue
    ###########    
    """
    positions.append(p)

all_pos = positions + lp + lm

close_pos = []
med_pos = []
long_pos = []

for par,dist in data_pos['distancias'].items():
    orig, dest = par.split('_')  
    if dist == 'C':
        if orig[:2] + orig[3:] == dest[:2] + dest[3:]:
            if orig[2] in ['A','Y']:
                close_pos.append(('{}-{}'.format(orig, 2), '{}-{}'.format(dest, 1)))
                close_pos.append(('{}-{}'.format(orig, 2), '{}-{}'.format(dest, 2)))
                close_pos.append(('{}-{}'.format(orig, 2), '{}-{}'.format(dest, 3)))
                close_pos.append(('{}-{}'.format(orig, 2), '{}-{}'.format(dest, 4)))
                close_pos.append(('{}-{}'.format(orig, 3), '{}-{}'.format(dest, 3)))
                close_pos.append(('{}-{}'.format(orig, 3), '{}-{}'.format(dest, 4)))
            elif orig[2] in ['B','X']:       
                close_pos.append(('{}-{}'.format(orig, 2), '{}-{}'.format(dest, 1)))
                close_pos.append(('{}-{}'.format(orig, 3), '{}-{}'.format(dest, 1)))  
                close_pos.append(('{}-{}'.format(orig, 3), '{}-{}'.format(dest, 2)))  
                close_pos.append(('{}-{}'.format(orig, 4), '{}-{}'.format(dest, 1)))  
                close_pos.append(('{}-{}'.format(orig, 4), '{}-{}'.format(dest, 2)))
                close_pos.append(('{}-{}'.format(orig, 4), '{}-{}'.format(dest, 3)))  
                
        else:
            for i in range(1,5):
                for j in range(1,5):
                    close_pos.append(('{}-{}'.format(orig, i), '{}-{}'.format(dest, j)))  
    elif dist == 'M':
        for i in range(1,5):
            for j in range(1,5):
                med_pos.append(('{}-{}'.format(orig, i), '{}-{}'.format(dest, j)))  
    elif dist == 'L':
        for i in range(1,5):
            for j in range(1,5):
                long_pos.append(('{}-{}'.format(orig, i), '{}-{}'.format(dest, j)))  


##### TEST 3/3 BORRAR
'''
fil= []
for i,j in close_pos:
    if i in positions and j in positions:
        fil.append((i,j))

close_pos = fil

fil= []
for i,j in med_pos:
    if i in positions and j in positions:
        fil.append((i,j))

med_pos = fil

fil= []
for i,j in long_pos:
    if i in positions and j in positions:
        fil.append((i,j))

long_pos = fil
file = open('check_pos', 'w+')
for i in range(0,len(positions),4):
    file.write('{},{},{},{}\n'.format(positions[i],positions[i+1],positions[i+2],positions[i+3]))
file.close()
file = open('check_c', 'w+')
for i in close_pos:
    file.write('{},{}\n'.format(i[0], i[1]))
file.close()
file = open('check_m', 'w+')
for i in med_pos:
    file.write('{},{}\n'.format(i[0], i[1]))
file.close()
file = open('check_l', 'w+')
for i in long_pos:
    file.write('{},{}\n'.format(i[0], i[1]))
file.close()
'''
print(len(close_pos))
print(len(med_pos))
print(len(long_pos))


12416
37888
534528


In [3]:
for c in Y0.keys():
    if c[0] in c_fijos:
        fix_pos.append(c[1])
            
for p in positions:
    if p not in fix_pos:
        mov_pos.append(p)

print("Resumen Posiciones")
print("-------------------------")
print("Total   (positions) :{:4}".format(len(positions)))
print("Fijas   (fix_pos)   :{:4}".format(len(fix_pos)))
print("Mobiles (mov_pos)   :{:4}".format(len(mov_pos)))
print("Llegada (lp)        :{:4}".format(len(lp)))
print("Salida  (lm)        :{:4}".format(len(lm)))
print("-------------------------")

Resumen Posiciones
-------------------------
Total   (positions) : 768
Fijas   (fix_pos)   : 572
Mobiles (mov_pos)   : 196
Llegada (lp)        :   1
Salida  (lm)        :   1
-------------------------


In [4]:
# Armar movimientos posibles

mov_cortos = []
mov_medios = []
mov_largos = []
# Filtrar todos los movimientos que parten o terminan en un p fijo

for o,d in close_pos:
    if o not in fix_pos:
        if d not in fix_pos:
            mov_cortos.append((o,d))
            
for o,d in med_pos:
    if o not in fix_pos:
        if d not in fix_pos:
            mov_medios.append((o,d))
            
for o,d in long_pos:
    if o not in fix_pos:
        if d not in fix_pos:
            mov_largos.append((o,d))

In [5]:

print("Resumen Movimientos Internos")
print("----------------------------------------")
print("Total                         :{:6}".format(len(close_pos) + len(med_pos) + len(long_pos)))
print("Total Filtrado                :{:6}".format(len(mov_cortos) + len(mov_medios) + len(mov_largos)))
print("----------------------------------------")
print("Cortos  (close_pos)           :{:6}".format(len(close_pos)))
print("Cortos Filtrados (mov_cortos) :{:6}".format(len(mov_cortos)))
print("Medios  (med_pos)             :{:6}".format(len(med_pos)))
print("Medios Filtrados (mov_medios) :{:6}".format(len(mov_medios)))
print("Largos  (long_pos)            :{:6}".format(len(long_pos)))
print("Largos Filtrados (mov_cortos) :{:6}".format(len(mov_largos)))
print("----------------------------------------")
print("posiciones x posiciones       :{:6}".format(len(positions)*len(positions)))
print("mov_pos x mov_pos             :{:6}".format(len(mov_pos)*len(mov_pos)))
print("----------------------------------------")

Resumen Movimientos Internos
----------------------------------------
Total                         :584832
Total Filtrado                : 37786
----------------------------------------
Cortos  (close_pos)           : 12416
Cortos Filtrados (mov_cortos) :   800
Medios  (med_pos)             : 37888
Medios Filtrados (mov_medios) :  1974
Largos  (long_pos)            :534528
Largos Filtrados (mov_cortos) : 35012
----------------------------------------
posiciones x posiciones       :589824
mov_pos x mov_pos             : 38416
----------------------------------------


In [6]:
# Crear U, B y B-1
#XXSBB-N
for p in positions:
    bloque = p[:2]
    fila = p[2:3]
    columna = p[p.find(fila)+1:p.find('-')] 
    nivel = p[-1:]
    # U
    if int(nivel) > 1:
        U[p] = '{}{}'.format(p[:-1],int(nivel)-1)


    # B
    temp = []
    if fila in ['A','Y']:
        for r in range(int(nivel)+1,5):
            temp.append('{}{}{}-{}'.format(bloque,fila,columna,r))
    if fila in ['B','X']:
        for r in range(int(nivel)+1,5):
            temp.append('{}{}{}-{}'.format(bloque,fila,columna,r))
        if fila == 'B':
            for r in range(int(nivel),5):
                if r > 1:
                    temp.append('{}{}{}-{}'.format(bloque,'A',columna,r))
        else:
            for r in range(int(nivel),5):
                if r > 1:
                    temp.append('{}{}{}-{}'.format(bloque,'Y',columna,r))
    if temp:
        B[p] = temp
    
#B-1
for i in B:
    for j in B[i]:
        if j not in B_I:
            B_I[j] = set([i])
        else:
            B_I[j].add(i)

print('====== U =======')
#for p in positions:
#    if p in U.keys():
#        print('{:5} : {}'.format(p, U[p]))


print('====== B =======')
#[print('{:7}:{} '.format(p,B[p])) for p in positions if p in B.keys()]


print('====== B-1 =======')
#[print('{:7}:{} '.format(p,B_I[p])) for p in positions if p in B_I.keys()]

====== U =======
====== B =======
====== B-1 =======


In [7]:
# Calculo de ETAPAS
e = 0
for c in c_salen:
    ep = 1
    for key in Y0.keys():
        if c == key[0],:
            if key[1] in B.keys():
                ep += len(B[key[1]])
    e += ep
etapas = list(range(1, e + len(c_llegan)+1))


print("Total Etapas (e) :{:4}".format(len(etapas)))


Total Etapas (e) :  68


In [39]:
#Calcular la etapa en la que cada contenedor se deberia mover
e = 0
etapas_activas = {}
etapas_inactivas = {}
e_llega = {}
e_sale = {}
for c in c_mueven:
    r_activo = []
    r_inactivo = []
    if c in c_salen:
        tipo = 'S'
        Top = T[c]
        ep = 1
        bloquea_a_otro_que_sale_antes = 0
        for key in Y0.keys():
            if c == key[0]:
                if key[1] in B.keys():
                    ep += len(B[key[1]])
                    break
        e += ep
        e_sale[c] = e
        #Reviso si esta bloqueando a otro que sale
        if key[1] in B_I.keys(): 
            for b in B_I[key[1]]:
                for k2 in Y0.keys():
                    if b == k2[1]:
                        if data['contenedores'][k2[0]][2] == 'sale':
                            if int(T[c]) > int(data['contenedores'][k2[0]][0]):
                                bloquea_a_otro_que_sale_antes = 1
                    
        if bloquea_a_otro_que_sale_antes == 0:
            [r_inactivo.append(i) for i in range(e,len(etapas)+1)]
            #[r_activo.append(i)   for i in range(1,e)]
        else:
            [r_activo.append(i) for i in range(1,e) ]
            [r_inactivo.append(i) for i in range(e,len(etapas)+1)]
    elif c in c_llegan:
        tipo = 'L'
        Top = T_In[c]
        e += 1
        e_llega[c] = e
        [r_inactivo.append(i) for i in range(1,e+1)]
        [r_activo.append(i)   for i in range(e+1,len(etapas)+1)]
    else:
         print(c)

#    print("{:13} {:2} {:3}".format(c, tipo, e))
    etapas_activas[c] = r_activo
    etapas_inactivas[c] = r_inactivo
    

Esta en [130115, 'RUX5-3', 'sale']
Esta en [130158, 'LUB9-4', 'sale']
Esta en [130171, 'RUX1-3', 'sale']
Esta en [130206, 'RUA11-2', 'sale']
Esta en [130208, 'RUY4-3', 'sale']
Esta en [130232, 'RDA8-3', 'sale']
Esta en [130241, 'RDX2-4', 'sale']
Esta en [130245, 'RUB8-3', 'sale']
Esta en [130247, 'LUB11-1', 'sale']
Esta en [130256, 'RUB12-4', 'sale']
Esta en [130268, 'RDY1-4', 'sale']
Esta en [130280, 'LDA5-3', 'sale']
Esta en [130281, 'LUA10-3', 'sale']
Esta en [130314, 'RDB5-3', 'sale']
Esta en [130319, 'LUY7-4', 'sale']


In [33]:
for c in c_salen:
     print('{} {}\nActivos: {}\nInactivos: {}'.format(c, e_sale[c], etapas_activas[c], etapas_inactivas[c]))
# for c in c_llegan:
#    print('{} {}\nActivos: {}\nInactivos: {}'.format(c, e_llega[c], etapas_activas[c], etapas_inactivas[c]))

C89-4304-90 61
Activos: []
Inactivos: [61, 62, 63, 64, 65, 66, 67, 68]
C77-3772-90 16
Activos: []
Inactivos: [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]
C62-3006-90 51
Activos: []
Inactivos: [51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]
C89-4319-90 68
Activos: []
Inactivos: [68]
C55-2695-90 28
Activos: []
Inactivos: [28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]
C79-3846-90 44
Activos: []
Inactivos: [44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]
C88-4282-90 56
Activos: []
Inactivos: [56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]
C63-3086-90 21
Activos: []
Inactivos: [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 

In [10]:
# Creo las tuplas para cada variable x(origen, destino, cont, etapa)

# Listado de movimientos internos x(origen, destino, cont, etapa)
t_int_close = []
t_int_medium = []
#t_int_long = []

for c in c_libres:
    for e in etapas[:85]:
        for o,d in mov_cortos:
            t = (o,d,c,e)
            t_int_close.append(t)
        for o,d in mov_medios:
            t = (o,d,c,e)
            t_int_medium.append(t)
#        for o,d in mov_largos:
#            t = (o,d,c,e)
#            t_int_long.append(t)

for c in c_mueven[:85]:
    for e in etapas_activas[c]:
        for o,d in mov_cortos:
            t = (o,d,c,e)
            t_int_close.append(t)
        for o,d in mov_medios:
            t = (o,d,c,e)
            t_int_medium.append(t)
#        for o,d in mov_largos:
#            t = (o,d,c,e)
#            t_int_long.append(t)

print('Resumen Tuplas (Variables X)')
print('-----------------------')
print('t_int_close  : {:8}'.format(len(t_int_close)))
print('t_int_medium : {:8}'.format(len(t_int_medium)))
#print('t_int_long   : {:8}'.format(len(t_int_long)))
print('-----------------------')

#Listado de movimientos cont que llegan x(dest, cont, etapa)
t_llegan = []
for c in c_llegan:
    for d in mov_pos:
        t = (d, c, e_llega[c])
        t_llegan.append(t)

t_salen = []
for c in c_salen:
    for o in mov_pos:
        t = (o, c, e_sale[c])
        t_salen.append(t)
        
print('t_llegan     : {:8}'.format(len(t_llegan)))
print('t_salen      : {:8}'.format(len(t_salen)))
print('-----------------------')

t_int_close  :   217600
t_int_medium :   536928
Resumen Tuplas (Variables)
-----------------------
t_int_close  :  1100000
t_int_medium :  2714250
-----------------------
t_llegan     :     5096
t_salen      :     2940
-----------------------


In [32]:
print(c_salen)

['C89-4304-90', 'C77-3772-90', 'C62-3006-90', 'C89-4319-90', 'C55-2695-90', 'C79-3846-90', 'C88-4282-90', 'C63-3086-90', 'C86-4192-90', 'C79-3851-90', 'C87-4231-90', 'C82-3979-90', 'C73-3584-90', 'C88-4260-90', 'C68-3312-90']


In [44]:
# Tuplas para las variables de posicion

# Para los fijos, solo tienen una posición posible
# Para lo que salen, estan hasta que salen
t_y_fijos = []
t_y_salen = []
for c,p in  Y0.keys():
    if c in c_fijos:
        [t_y_fijos.append((p, c, e)) for e in etapas]
    elif c in c_salen:
        [t_y_salen.append((p, c, e)) for e in etapas if e < e_sale[c] or e in etapas_activas[c]]
        


# Para los que llegan pueden estar en cualquier posicion no fija despues de que arriban
t_y_llegan = []
for c in c_llegan:
    for p in mov_pos:
        [t_y_llegan.append((p, c, e)) for e in etapas_activas[c]]

# Para los libres pueden estar en cualquier parte en todo momento
t_y_libres = []
for c in c_libres:
    for p in mov_pos:
        [t_y_libres.append((p, c, e)) for e in etapas]

print('Resumen Tuplas (Variables Y)')
print('-----------------------')
print('t_y_fijos     : {:8}'.format(len(t_y_fijos)))
print('t_y_salen     : {:8}'.format(len(t_y_salen)))
print('t_y_llegan    : {:8}'.format(len(t_y_llegan)))
print('t_y_libres    : {:8}'.format(len(t_y_libres)))
print('-----------------------')
print('Total (Y)     : {:8}'.format(len(t_y_fijos) + len(t_y_salen) + len(t_y_llegan)+len(t_y_libres)))
print('-----------------------')

Resumen Tuplas (Variables Y)
-----------------------
t_y_fijos     :    38896
t_y_salen     :      631
t_y_llegan    :   216188
t_y_libres    :    53312
-----------------------
Total (Y)     :   309027
-----------------------


In [11]:
######### MODELO ########

m = gp.Model()

Academic license - for non-commercial use only - expires 2021-04-12
Using license file /home/jpcavada/gurobi.lic


In [49]:
###### VARIABLES #####

#x Movimientos entre el patio
print('Creando Variables')
print('-----------------------')
print("XC   ", end="")
x_c = m.addVars(t_int_close, vtype=GRB.BINARY, name='XC')
print("{:7} ({:8.2f} MB)".format(len(x_c), sys.getsizeof(x_c)/(1024**2)))

print("XM   ", end="")
x_m = m.addVars(t_int_medium, vtype=GRB.BINARY, name='XM')
print("{:7} ({:8.2f} MB)".format(len(x_m), sys.getsizeof(x_m)/(1024**2)))
#x_l = m.addVars(t_int_long, vtype=GRB.BINARY, name='XL')

#v Movimientos de llegada (entre l- y p)
print("V    ", end="")
v = m.addVars(t_llegan, vtype=GRB.BINARY, name='V')
print("{:7} ({:8.2f} MB)".format(len(v), sys.getsizeof(v)/(1024**2)))

#w Movimientos de salida (entre p y l+)
print("W    ", end="")
w = m.addVars(t_salen, vtype=GRB.BINARY, name='W')
print("{:7} ({:8.2f} MB)".format(len(w), sys.getsizeof(w)/(1024**2)))
#x = m.addVars(all_pos, all_pos, c_mueven, etapas, vtype=GRB.BINARY, name='X')


# Posicion de cada contenedor y(pos, c, e)
print("Y    ", end="")
y = m.addVars(t_y_fijos + t_y_salen + t_y_llegan + t_y_libres, vtype=GRB.BINARY, name='Y')
print("{:7} ({:8.2f} MB)".format(len(y), sys.getsizeof(y)/(1024**2)))

# Movimientos z(e)
print("z    ", end="")
z = m.addVars(etapas, vtype=GRB.BINARY, name='Z')
print("{:7} ({:8.2f} MB)".format(len(z), sys.getsizeof(z)/(1024**2)))

Creando Variables
-----------------------
XC   1100000 (   40.00 MB)
XC   2714250 (   80.00 MB)
V       5096 (    0.14 MB)
W       2940 (    0.14 MB)
Y     309027 (   10.00 MB)
z         68 (    0.00 MB)


In [ ]:
            
##### Objective (C1)#####

m.setObjective(y.sum('*','*','*') + x.sum('*','*','*','*') + r.sum('*'), GRB.MINIMIZE)

##### CONTRAINTS #######

#C2: Conservacion de Flujo
for c in c_mueven:
    for e in etapas[1:]:
        for i in all_pos:
            m.addConstr(y[i,c,e] == y[i,c,e-1] 
                                    - (gp.quicksum(x[i,j,c,e] for j in all_pos if j != i))
                                    + (gp.quicksum(x[j,i,c,e] for j in all_pos if j != i))
                                    ,name = "c2")

#C3: Pocision Inicial contenedores
for key, val in Y0.items():
    m.addConstr(y[key[1],key[0],0] == Y0[key[0],key[1]], name='c3')

In [ ]:
#C4: Un contenedor por posicion
for i in positions:
    for e in etapas:
        m.addConstr((gp.quicksum(y[i,c,e] for c in cont)) <= 1, name='c4')
        
#C5: Solo pueden salir de l+ los que llegan
for c in c_mueven:
    if c not in c_llegan:
        for e in etapas:
            for j in all_pos:
                aux_constr = m.addConstr(x['arrival',j,c,e] == 0, name='c5')


#C6: Solo pueden entrar a l- los que se van
for c in c_mueven:
    if c not in c_salen:
        for e in etapas:
            for i in all_pos:
                aux_constr =  m.addConstr(x[i,'retrival',c,e] == 0, name='c6')
                
#C7: Nadie entra a l+
for c in c_mueven:
    for e in etapas:
        for i in all_pos:
            aux_contrs = m.addConstr(x[i,'arrival',c,e] == 0, name='c7')
        

In [ ]:
#C8: Nadie sale de l-
for c in cont:
    for e in etapas:
        for j in all_pos:
            aux_constr = m.addConstr(x['retrival',j,c,e] == 0, name='c8')
            aux_constr.setAttr("Lazy", 1)

#C9: Un movimiento por etapa
for e in etapas:
    m.addConstr((gp.quicksum(x[i,j,c,e] for i in all_pos for j in all_pos for c in cont))<=1, name='c9')
    

#C10: Prohibido salir de posiciones bloqueadas
for i in all_pos:
    if i in B.keys():
        for p in B[i]:
            for e in etapas:
                aux_constr = m.addConstr(gp.quicksum(x[i,j,c,e] for j in all_pos for c in cont) <= 1 - (gp.quicksum(y[p,c,e] for c in cont)), name='c10')
                aux_constr.setAttr("Lazy", 1)
                

In [ ]:
#C11: No puedo llegar a posiciones bloquedas
for j in all_pos:
    if j in B.keys():
        for p in B[j]:
            for e in etapas:
                aux_constr = m.addConstr(gp.quicksum(x[i,j,c,e] for i in all_pos for c in cont) <= 1 - (gp.quicksum(y[p,c,e] for c in cont)), name='c11')
                aux_constr.setAttr("Lazy", 1)
                
#C12: No puedo llegar al aire
for j in all_pos:
    if j in U.keys():
            for e in etapas:
                aux_constr = m.addConstr(gp.quicksum(x[i,j,c,e] for i in all_pos for c in cont) <= y[U[j],c,e], name='c12' )
                aux_constr.setAttr("Lazy", 1)
                
#C13: Tiene que entrar todos los que llegan:
m.addConstr(gp.quicksum(y['arrival',c,etapas[-1]] for c in c_llegan)==0, name='c13')

#C14: Tiene que salir todos los que salen
m.addConstr(gp.quicksum(y['retrival',c,etapas[-1]] for c in c_salen)==len(c_salen), name='c14')

In [ ]:
#C15: Orden de las operaciones
for c in range(0,len(c_mueven)-1):
    for e in etapas:
        m.addConstr(
                    (gp.quicksum(
                                gp.quicksum(x['arrival',j,c_mueven[c],t] for j in positions) + gp.quicksum(x[i,'retrival',c_mueven[c],t] for i in positions)
                                for t in etapas if t <= e-1)) 
                    >= 
                    (gp.quicksum(x['arrival',j,c_mueven[c+1],e] for j in positions) + gp.quicksum(x[i,'retrival',c_mueven[c+1],e] for i in positions)),
            name="c15"
        )

##############  RESTRICCIONES DE RELOCACIONES ################
#C19: Definir R
for j in positions:
    if j in B_I.keys():
        for c in c_mueven:
            for e in etapas:
                m.addConstr(
                    len(B_I[j])*(r[e]+1) >=
                    (gp.quicksum( 
                        (gp.quicksum(y[k,cp,e] for cp in cont if T[cp] <= T[c]) + 
                                         len(B_I[j])*(gp.quicksum(x[i,j,c,e] for i in all_pos))
                                                        ) for k in B_I[j])
                                        )
                    ,name='c19')

In [ ]:
############## PARAMETRIZACION Y EJECUCION ################                
m.Params.MIPGap = 0.1
m.optimize()

In [ ]:
for p in all_pos:
    print(p)